In [2]:
import numpy as np
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.wcs import WCS
import os
from scipy import optimize, stats, interpolate
from scipy.integrate import quad
import argparse
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatter
from matplotlib import cm
import sys
import pandas as pd
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
import healpy as hp
from copy import copy, deepcopy
# sys.path.append('BatML/')
import logging, traceback
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [3]:
%cd BatML/

/storage/work/j/jjd330/local/bat_data/BatML


In [4]:
from config import solid_angle_dpi_fname, fp_dir, rt_dir
from ray_trace_funcs import RayTraces
from event2dpi_funcs import det2dpis, mask_detxy
# from do_InFoV_scan3 import Swift_Mask_Interactions, Source_Model_InFoV, Bkg_Model_wFlatA,\
#                             CompoundModel, Point_Source_Model_Binned_Rates,\
#                             theta_phi2imxy, bldmask2batxys, imxy2theta_phi,\
#                             get_fixture_struct
from flux_models import Cutoff_Plaw_Flux, Plaw_Flux, get_eflux_from_model
from models import Source_Model_InOutFoV, Bkg_Model_wFlatA, CompoundModel, Point_Source_Model_Binned_Rates
from ray_trace_funcs import RayTraces
from sqlite_funcs import get_conn
from dbread_funcs import get_info_tab, get_twinds_tab
from do_manage import im_dist
from hp_funcs import ang_sep
from coord_conv_funcs import theta_phi2imxy, imxy2theta_phi, convert_imxy2radec,\
                            convert_radec2thetaphi, convert_radec2imxy
from do_llh_inFoV4realtime import parse_bkg_csv
from LLH import LLH_webins
from minimizers import NLLH_ScipyMinimize_Wjacob, NLLH_ScipyMinimize

In [5]:
def get_bldmask_alldets():

    detxs_by_sand0 = np.arange(0, 286-15, 18)
    detxs_by_sand1 = detxs_by_sand0 + 15

    detys_by_sand0 = np.arange(0, 173-7, 11)
    detys_by_sand1 = detys_by_sand0 + 7

    all_good_detxs = np.ravel([np.arange(detxs_by_sand0[i], detxs_by_sand1[i]+1,\
                                          1, dtype=np.int) for i in range(16)])
    all_good_detys = np.ravel([np.arange(detys_by_sand0[i], detys_by_sand1[i]+1,\
                                         1, dtype=np.int) for i in range(16)])

    detxax = np.arange(286, dtype=np.int)
    detyax = np.arange(173, dtype=np.int)
    detx_dpi, dety_dpi = np.meshgrid(detxax, detyax)
    bl_alldets = np.isin(detx_dpi, all_good_detxs)&np.isin(dety_dpi, all_good_detys)
    return bl_alldets

bl_alldets = get_bldmask_alldets()

In [1]:
dmask = fits.open('detmask.fits')[0].data
ndets = np.sum(dmask==0)
bl_dmask = (dmask==0.)

NameError: name 'fits' is not defined

In [7]:
# use whatever ebins
ebins0 = np.array([15.0, 24.0, 35.0, 48.0, 64.0])
ebins0 = np.append(ebins0, np.logspace(np.log10(84.0), np.log10(500.0), 5+1))[:-1]
ebins0 = np.round(ebins0, decimals=1)[:-1]
ebins1 = np.append(ebins0[1:], [350.0])
nebins = len(ebins0)

In [8]:
flux_mod = Cutoff_Plaw_Flux(E0=100.0)
flux_params = {'A':1.0, 'gamma':0.5, 'Epeak':1e2}
rt_obj = RayTraces(rt_dir)

INFO:root:0.02
INFO:root:0.002


In [10]:
drm_fname = '' # an already made DRM to copy the header info from

In [9]:
%%time

theta = 0.0
phi = 0.0

sig_mod = Source_Model_InOutFoV(flux_mod, [ebins0,ebins1], bl_dmask,\
                                rt_obj, use_deriv=True)    

sig_mod.set_theta_phi(theta, phi)

print('Theta phi set')

resps = np.sum(sig_mod.resp_obj.tot_resp_dpis, axis=0)

dic = {'ENERG_LO':sig_mod.resp_obj.PhotonEmins, 
       'ENERG_HI':sig_mod.resp_obj.PhotonEmaxs,
       'N_GRP':np.ones(187, dtype=np.int),
       'F_CHAN':np.zeros(187, dtype=np.int),
       'N_CHAN':np.zeros(187, dtype=np.int) + len(ebins0),
       'MATRIX':resps}
drm_tab = Table(data=dic)

ebounds_tab = Table(data=[np.arange(len(ebins0),dtype=np.int),ebins0,ebins1],
                    names=['CHANNEL', 'E_MIN', 'E_MAX'], dtype=[np.int,np.float,np.float])

primary_hdu = fits.PrimaryHDU()
drm_hdu = fits.table_to_hdu(drm_tab)
ebounds_hdu = fits.table_to_hdu(ebounds_tab)

drm_hdu.name = 'SPECRESP MATRIX'
ebounds_hdu.name = 'EBOUNDS'

f0 = fits.open(drm_fname)

for k, val in f0[1].header.iteritems():
    if k in drm_hdu.header.keys():
        continue
    drm_hdu.header.append((k,val))
drm_hdu.header['DETCHANS'] = len(ebins0)
drm_hdu.header['TLMAX4'] = 4
drm_hdu.header['NUMGRP'] = len(drm_tab['ENERG_HI'])
drm_hdu.header['NUMELT'] = len(drm_tab['ENERG_HI'])*len(ebins0)


for k, val in f0[2].header.iteritems():
    if k in ebounds_hdu.header.keys():
        continue
    ebounds_hdu.header.append((k,val))
ebounds_hdu.header['DETCHANS'] = len(ebins0)


hdul = fits.HDUList([primary_hdu, drm_hdu, ebounds_hdu])

save_fname = 'DRM.rsp'

# hdul.writeto(save_fname, overwrite=True)

INFO:root:Making new response object
(0.0, 30.48, -14.117)


(0.0, 30.48, -14.117)


response.py:1499: RuntimeWarning: invalid value encountered in divide
  (self.orig_photonEs[self.Einds1] - self.orig_photonEs[self.Einds0])
Polygons.py:317: RuntimeWarning: invalid value encountered in less
  bl_goodz[((z-detz) < (-z_height/2.))] = False


0.0 3.0
0.0
0.0
DEBUG:root:opening file: fwd_ray_trace_-0.02000_0.00000_0.00000_0.02000_.npy
DEBUG:root:nbytes_total=95789408.0
max rt: 1.0000


shield_structure.py:239: RuntimeWarning: invalid value encountered in less
  bl_goodz[((z-detz) < -0.2)] = False


DEBUG:root:Openning file: resp_by_sand_hpind_40.fits
DEBUG:root:Openning file: resp_by_sand_hpind_55.fits
DEBUG:root:Openning file: resp_by_sand_theta_0.fits
initing ResponseDPI, with fname
/storage/work/jjd330/local/bat_data/resp_tabs/drm_theta_0.0_phi_0.0_.fits
Theta phi set


NameError: name 'drm_fname' is not defined